The NOAA HRRR is a real-time 3km resolution, hourly updated, cloud-resolving, convection-allowing atmospheric model, initialized by 3km grids with 3km radar assimilation.

This notebook provides an example of accessing HRRR data, including (1) finding the data file corresponding to a date and time, (2) retrieving a portion of that file from blob storage which includes the surface temperature variable, (3) opening the file using the xarray library, and (4) rendering an image of the forecast.

This dataset is documented at http://aka.ms/ai4edata-hrrr.

In [1]:
import climatedata_functions as climf
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
metadata = pd.read_csv("../data/metadata.csv")

In [3]:
ds = climf.get_ds()#only words with recent dates

### Make metadafile with gridpoints

In [4]:
#tranform coordinates for longitude
metadata['longitude_trans'] = metadata['longitude']+360 
#add columns for gridpoints
metadata['x_grid'] = ''
metadata['y_grid'] = ''

#make new metadatafile and save as cvs 
#metadata_new = climf.save_grids(metadata.head(), ds) #(takes approx 6 minutes)
#metadata_new.to_csv('../data/metadata_grids.csv')


In [5]:
#load new metadata file that includes the gridpoints
metadata = pd.read_csv("../data/metadata_grids.csv")

In [6]:
metadata.head(10)

,Unnamed: 0,uid,latitude,longitude,date,split,longitude_trans,x_grid,y_grid
0,0,aabm,39.080319,-86.430867,2018-05-14,train,273.569133,570,1217
1,1,aabn,36.559700,-121.510000,2016-08-31,test,238.490000,550,192
2,2,aacd,35.875083,-78.878434,2020-11-19,train,281.121566,488,1455
3,3,aaee,35.487000,-79.062133,2016-08-24,train,280.937867,473,1453
4,4,aaff,38.049471,-99.827001,2019-07-23,train,260.172999,513,831
5,5,aafl,39.474744,-86.898353,2021-08-23,train,273.101647,583,1202
6,6,aafp,35.647742,-79.271782,2017-11-15,train,280.728218,478,1445
7,7,aagm,35.906885,-79.132962,2020-06-10,train,280.867038,488,1447
8,8,aahy,35.726522,-79.125458,2014-08-12,train,280.874542,481,1449
9,9,aaia,35.980000,-78.791686,2018-06-27,train,281.208314,493,1457


### Get temperatures for gridpoints

Make function that gets the mean/median temperature for one date for all the places sampled at that date (to save time) and the previos x days

In [16]:
date = metadata.date[0]
done_list = [] #make list where date that was already sampled is appended


date_index_list = metadata.index[metadata.date == date]
done_list.append(date)
done_list

for idx in date_index_list:


['2018-05-14']

In [33]:
temp_array = [[0] * 7 for i in range(3)]#7 tage, 3 ids
temp_array

[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]

In [32]:
temp_array[:][0]###erst liste

[2, 2]

In [24]:
type(temp_array[1])

list

In [32]:
from datetime import timedelta


start_date = climf.get_start_date(metadata.date[0])
date_index_list = metadata.index[metadata.date == start_date]
#print(date_index_list)
temp_list = []
done_list = []
days = 2
for idx,row in enumerate(metadata.index): #takes first element in metadata list
    print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds = climf.get_ds_aws(day_date)#getting the temperature array for the specified date
            for index in date_index_list:
                print(x,count)
                x = metadata.x_grid[index]
                y = metadata.y_grid[index]
                #temp_list[x][count] = ds[x][y]
                count += 1
            print(temp_list)
        if idx == 1:
            break  
    else:
        continue 

0 0
Int64Index([0, 3011, 4619, 5926, 6323, 8822, 9355, 12844, 13422, 15238, 15598,
            23349],
           dtype='int64')
12 2
0 0
570 1
595 2
561 3
536 4
549 5
594 6
601 7
592 8
567 9
561 10
532 11
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
1 0
570 1
595 2
561 3
536 4
549 5
594 6
601 7
592 8
567 9
561 10
532 11
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
1 1
Int64Index([    1,   231,   505,   693,   833,  1045,  1149,  1280,  1418,
             1760,  1919,  2243,  2594,  2899,  3172,  3266,  3647,  3677,
             3806,  3826,  3877,  3939,  4053,  4146,  5058,  5136,  5372,
             5780,  6058,  6228,  7108,  7371,  7488,  7525,  7754,  7761,
             8097,  8658,  9125,  9327,  9570, 10347, 10369, 10848, 11067,
            11260, 12807, 13050, 13322, 13345, 14317, 14320, 15019, 15040,
            15700, 15920, 16700, 16777, 16936, 17389, 17523, 18110, 18310,

In [33]:
metadata.head()

,Unnamed: 0,uid,latitude,longitude,date,split,longitude_trans,x_grid,y_grid
0,0,aabm,39.080319,-86.430867,2018-05-14,train,273.569133,570,1217
1,1,aabn,36.559700,-121.510000,2016-08-31,test,238.490000,550,192
2,2,aacd,35.875083,-78.878434,2020-11-19,train,281.121566,488,1455
3,3,aaee,35.487000,-79.062133,2016-08-24,train,280.937867,473,1453
4,4,aaff,38.049471,-99.827001,2019-07-23,train,260.172999,513,831


In [36]:
start_date = metadata.date[0]
date_index_list = metadata.index[metadata.date == start_date]
date_index_list.index == date_index_list

Int64Index([0, 3011, 4619, 5926, 6323, 8822, 9355, 12844, 13422, 15238, 15598,
            23349],
           dtype='int64')

In [66]:
metadata.y_grid

0        1217
1         192
2        1455
3        1453
4         831
         ... 
23565     187
23566     829
23567    1452
23568    1446
23569     941
Name: y_grid, Length: 23570, dtype: int64

In [67]:
print(x,y)
#ds[x][y].values

0    570
Name: x_grid, dtype: int64 0    1217
Name: y_grid, dtype: int64


In [55]:
ds

<xarray.DataArray 't' (y: 1059, x: 1799)>
[1905141 values with dtype=float32]
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (y, x) float64 ...
    longitude   (y, x) float64 ...
    valid_time  datetime64[ns] ...
Dimensions without coordinates: y, x
Attributes: (12/33)
    GRIB_paramId:                             130
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1905141
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [18]:
from datetime import timedelta
start_date = climf.get_start_date(metadata.date[0])
temp_list = []
days = 2

for row in range(len(metadata[0].uid)):
    start_date = metadata.date[row]
    date_index_list = metadata.index[metadata.date == date]
    done_list.append(start_date)
    start_date = climf.get_start_date(start_date)
    temp_array = [[0] * len(start_date) for i in range(days)]
    print(temp_array)
    for x in range(days):
        day_date = start_date - timedelta(days=x)
        print(day_date)
        ds = climf.get_temp_aws(day_date)##input changed
        for idx in date_index_list:
            

            temp_list.append(temperature)
    temp_list


metadata.x_grid[0] ,metadata.y_grid[0]

KeyError: 0